In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env

In [2]:
# get data for the exercise
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)

In [3]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)


In [4]:
# apply parse function
df = df.entry.apply(parse_log_entry)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [5]:
# check ip value counts
df.ip.value_counts().head()

97.105.19.58      11998
173.173.113.51     1059
72.181.113.170      613
72.181.105.81       246
24.26.242.9          21
Name: ip, dtype: int64

In [6]:
# create a probability function
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )

In [9]:
status_df = value_counts_and_frequencies(df.status_code)
status_df.head()

,count,proba
200,13958,0.998855
499,16,0.001145


In [13]:
status_given_ip = (
    df.groupby('status_code')
    .ip.value_counts(normalize=True)
    .rename('proba_ip_given_status_code')
    .reset_index()
)
status_given_ip.head()

,status_code,ip,proba_ip_given_status_code
0,200,97.105.19.58,0.858576
1,200,173.173.113.51,0.075870
2,200,72.181.113.170,0.043774
3,200,72.181.105.81,0.017624
4,200,24.26.242.9,0.001505


In [10]:
ip_df = value_counts_and_frequencies(df.ip)
ip_df.head()

,count,proba
97.105.19.58,11998,0.858595
173.173.113.51,1059,0.075784
72.181.113.170,613,0.043867
72.181.105.81,246,0.017604
24.26.242.9,21,0.001503


In [11]:
status_given_ip = (
    df.groupby('ip')
    .status_code.value_counts(normalize=True)
    .rename('proba_status_given_ip')
    .reset_index()
)
status_given_ip.head()

,ip,status_code,proba_status_given_ip
0,173.173.113.51,200,1.0
1,24.26.242.9,200,1.0
2,3.88.129.158,200,1.0
3,3.92.201.136,200,1.0
4,34.207.64.242,200,1.0


In [15]:
path_df = value_counts_and_frequencies(df.request_path)
path_df.head()

,count,proba
/api/v1/sales?page=2,709,0.050737
/api/v1/items,464,0.033205
/api/v1/items?page=2,291,0.020824
/api/v1/items?page=3,219,0.015672
/api/v1/stores,162,0.011593


In [16]:
status_given_ip = (
    df.groupby('request_path')
    .ip.value_counts(normalize=True)
    .rename('proba_ip_given_request_path')
    .reset_index()
)
status_given_ip.head()

,request_path,ip,proba_ip_given_request_path
0,/,97.105.19.58,0.887850
1,/,68.201.219.223,0.028037
2,/,72.181.105.81,0.028037
3,/,173.173.113.51,0.009346
4,/,3.92.201.136,0.009346


In [17]:
status_given_ip = (
    df.groupby('request_path')
    .status_code.value_counts(normalize=True)
    .rename('proba_status_code_given_request_path')
    .reset_index()
)
status_given_ip.head()

,request_path,status_code,proba_status_code_given_request_path
0,/,200,1.0
1,/api/V1/HiZach!,200,1.0
2,/api/v1,200,1.0
3,/api/v1/,200,1.0
4,/api/v1//api/v1/items,200,1.0


In [18]:
user_df = value_counts_and_frequencies(df.user_agent)
user_df.head()

,count,proba
python-requests/2.21.0,12001,0.858809
python-requests/2.20.1,1911,0.136754
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36",34,0.002433
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0,8,0.000572
Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots),7,0.000501


In [19]:
status_given_ip = (
    df.groupby('user_agent')
    .ip.value_counts(normalize=True)
    .rename('proba_ip_given_user_agent')
    .reset_index()
)
status_given_ip.head()

,user_agent,ip,proba_ip_given_user_agent
0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,72.181.105.81,1.000000
1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,70.121.214.34,1.000000
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,72.181.113.170,0.500000
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,97.105.19.58,0.500000
4,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,97.105.19.58,0.764706


In [20]:
status_given_ip = (
    df.groupby('user_agent')
    .request_path.value_counts(normalize=True)
    .rename('proba_request_path_given_user_agent')
    .reset_index()
)
status_given_ip.head()

,user_agent,request_path,proba_request_path_given_user_agent
0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,/api/v1/sales,0.250
1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,/,0.125
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,/api/v1/items?page=2,0.125
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,/api/v1/items?page=3,0.125
4,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,/api/v1/items?page=4,0.125
